In [1]:
import sys,pandas,numpy,os
import torch
from torch.cuda import FloatTensor
from torch.autograd import Variable
from torch import nn
from torch.nn import functional
sys.path.append('/home/heyangle/Desktop/ScanCraft/ScanCraft')
from command.scan.scan import scan
from command.pytorch.normalize import GetRanges
from command.pytorch.classify import Classify,BCEloss
from command.pytorch.estimate import Estimate,MSELoss
from command.pytorch.normalize import normalize#,NormalizeArray,DenormalizeArray
from command.pytorch.variable import CudaVariableFromArray, ArrayFromCudaVariable

mold=scan(method='random')
mold.AddScalar('tanB','MINPAR',3,1.,60.)
mold.AddScalar('Atop','EXTPAR'   ,11  ,  -6e3    ,6e3)
mold.AddFollower('Abottom','EXTPAR'   ,12,'Atop')
mold.AddScalar('Lambda','EXTPAR'  ,61  ,1e-3    ,1. ,prior_distribution='exponential')
mold.AddScalar('Kappa','EXTPAR'   ,62 ,1.e-3    ,1. ,prior_distribution='exponential')
mold.AddScalar('A_Lambda','EXTPAR' ,63,-3.e3,3.e3)
mold.AddScalar('A_kappa','EXTPAR' ,64,-3.e3,3.e3)
mold.AddScalar('mu_eff','EXTPAR'  ,65,100.,1500.)

Norm=normalize(mold)
print('Input mode been set')

Input mode been set


In [2]:
# IsCalculable=Classify(7,1000,500).cuda()
# c_opt=torch.optim.Adam(IsCalculable.parameters(),lr=0.01)
# EstimateMh=Estimate(7,500,300).cuda()
# e_opt=torch.optim.Adam(EstimateMh.parameters(),lr=0.001)
# print('NNs been set')

# '''arrays of original training samples and test samples'''
acc_train =pandas.read_csv('Pylon/accepted_train.csv',header=[0,1,2,3],index_col=0)
exc_train =pandas.read_csv('Pylon/excluded_train.csv',header=[0,1,2,3],index_col=0)
mh_train  =pandas.read_csv('Pylon/mass_train.csv',    header=[0,1,],   index_col=0)


# calculable
c_data_train  =numpy.vstack((acc_train.values,exc_train.values[:13718]))
c_inp_train   =Norm(c_data_train[:,:7])
c_target_train=c_data_train[:,-1].reshape((len(c_data_train),1))
in_CV=CudaVariableFromArray(c_inp_train)
target_CV=CudaVariableFromArray(c_target_train,requires_grad=False)

# acc_test  =pandas.read_csv('laboratory/Pylon/accepted_test.csv' ,header=[0,1,2,3],index_col=0)
# exc_test  =pandas.read_csv('laboratory/Pylon/excluded_test.csv' ,header=[0,1,2,3],index_col=0)
# mh_test   =pandas.read_csv('laboratory/Pylon/mass_test.csv'     ,header=[0,1,],   index_col=0)
# c_data_test   =numpy.vstack((acc_test.values,exc_test.values[:6632]))
# c_inp_test    =Norm(c_data_test[:,:7])
# c_target_test =c_data_test[:,-1].reshape((len(c_data_test),1))

# estimate Higgs mass
in_EV    =CudaVariableFromArray(Norm(acc_train.values[:,:-1]))
target_EV=CudaVariableFromArray(numpy.log10(mh_train.values))

In [13]:
for i in range(1000):
    pred=IsCalculable(in_CV)
    loss=BCEloss(pred,target_CV)
    if i%100==0:
        print(i,loss.data[0])
    c_opt.zero_grad()
    loss.backward()
    c_opt.step()
print(loss)

0 0.5673403739929199
100 0.5668604969978333
200 0.5661951899528503
300 0.5650193691253662
400 0.564426600933075
500 0.5641022324562073
600 0.5636773109436035
700 0.5634415149688721
800 0.5633980631828308
900 0.5633448362350464
Variable containing:
 0.5633
[torch.cuda.FloatTensor of size 1 (GPU 0)]



In [18]:
pred.data.cpu().numpy()

array([[ 0.99955851],
       [ 0.99918503],
       [ 0.99586982],
       ..., 
       [ 0.        ],
       [ 0.        ],
       [ 0.        ]], dtype=float32)

In [19]:
for i in range(1000):
    pred=EstimateMh(in_EV)
    loss=MSELoss(pred,target_EV)
    if i%100==0:
        print(i,loss.data[0])
    e_opt.zero_grad()
    loss.backward()
    e_opt.step()
print(loss)

0 0.0047939433716237545
100 0.004691573791205883
200 0.004593733698129654
300 0.004507002886384726
400 0.004413235932588577
500 0.004233905114233494
600 0.004033249337226152
700 0.003882306395098567
800 0.0037732140626758337
900 0.0036909659393131733
Variable containing:
1.00000e-03 *
  3.5912
[torch.cuda.FloatTensor of size 1 (GPU 0)]



In [4]:
# torch.save(IsCalculable,'IsCalculable')
# torch.save(EstimateMh,'EstimateMh')

In [3]:
# load models
IsCalculable=torch.load('IsCalculable')
EstimateMh=torch.load('EstimateMh')

In [3]:
# # re-training models
# acc=pandas.read_csv('out_in_the_past/output_0121/accepted.csv',header=[0,1,2,3],index_col=0)
# exc=pandas.read_csv('out_in_the_past/output_0121/excluded.csv',header=[0,1,2,3],index_col=0)
# mh =pandas.read_csv('out_in_the_past/output_0121/mass.csv',    header=[0,1,],   index_col=0)

In [8]:
# data  =numpy.vstack((acc.values,exc.values))
# in_CV    =CudaVariableFromArray(Norm(data[:,:7]))
# target_CV=CudaVariableFromArray(data[:,-1].reshape(len(data),1),requires_grad=False)
# in_EV    =CudaVariableFromArray(Norm(acc.values[:,:-1]))
# target_EV=CudaVariableFromArray(numpy.log10(mh.values))

In [9]:
# c_opt=torch.optim.Adam(IsCalculable.parameters(),lr=0.001)
# e_opt=torch.optim.Adam(EstimateMh.parameters(),lr=0.0001)

In [4]:
# generate new input to run in NMTools
new_tf=FloatTensor(0)
mh_0=FloatTensor([95.,125.,95.])
while len(new_tf)<8000:
    #1 million random input points
    random_tf=FloatTensor(100000,7)
    nn.init.uniform(random_tf,a=-1,b=1)
    pred_tf=IsCalculable(Variable(random_tf)).data
    calculable_tf=random_tf[torch.nonzero(pred_tf>0.5)[:,0]]# calculable input points
    lg_mh_tf=EstimateMh(Variable(calculable_tf)).data
    mh_tf=torch.pow(10.,lg_mh_tf)# higgs mass : h1, h2, A1
    D_mh=mh_tf-mh_0# mi - m0
    x2=torch.min(D_mh[:,0::2]**2,1)[0]+D_mh[:,1]**2
    D_x2=x2-torch.min(x2)
    like=torch.exp(-10.*D_x2/torch.mean(D_x2)) # rescaled likelyhood
    threshold=nn.init.uniform(torch.zeros_like(like))
    passengers=calculable_tf[torch.nonzero(like>threshold)[:,0]]
    new_tf=torch.cat((new_tf,passengers),dim=0)

# random points
additional=nn.init.uniform(FloatTensor(2000,7))
new_tf=torch.cat((new_tf,additional),dim=0)
new_array=Norm.D(new_tf.cpu().numpy())

In [21]:
new_array

array([[  3.34065487e+01,  -1.78097856e+03,   1.14706851e-01, ...,
         -2.66759437e+03,  -1.72237515e+03,   1.28701957e+03],
       [  5.72895797e+01,  -5.50138593e+03,   1.64020327e-01, ...,
          1.28730977e+03,  -5.69197297e+02,   1.44921396e+03],
       [  5.80161492e+01,  -1.36236763e+03,   5.96447186e-01, ...,
          2.98696053e+03,  -2.51109838e+02,   1.37226084e+03],
       ..., 
       [  5.61821158e+01,   2.60786247e+03,   5.75477269e-01, ...,
          1.93214607e+03,   2.96627283e+03,   1.45259187e+03],
       [  3.49791104e+01,   2.32970881e+03,   7.78535680e-01, ...,
          1.03331280e+03,   1.30863583e+03,   1.28076584e+03],
       [  5.49992403e+01,   4.17529821e+02,   5.70247487e-01, ...,
          2.08280182e+03,   1.70637918e+03,   1.36832609e+03]])

In [5]:
from command.multi_thread.MT_NTools import MT_NTools
from command.data_transformer.InputListToPandas import InputListToPandas as I2P
from command.data_transformer.ArrayToInputQueue import ArrayToInputQueue as a2Q
MTN=MT_NTools(threads=7)

delete folder ./output? (y/n) 



In [6]:
Queue=a2Q(new_array,mold)

In [7]:
MTN.Run(Queue)

Calculations begin at Tue Jan 23 15:12:28 2018
  threads:	7
  points:	10052
  thread-0	runing,    10000 points left at Tue Jan 23 15:12:32 2018
  thread-1	runing,     9000 points left at Tue Jan 23 15:13:50 2018
  thread-2	runing,     8000 points left at Tue Jan 23 15:15:16 2018
  thread-0	runing,     7000 points left at Tue Jan 23 15:16:34 2018
  thread-0	runing,     6000 points left at Tue Jan 23 15:17:59 2018
  thread-1	runing,     5000 points left at Tue Jan 23 15:19:36 2018
  thread-5	runing,     4000 points left at Tue Jan 23 15:21:28 2018
  thread-2	runing,     3000 points left at Tue Jan 23 15:22:46 2018
  thread-1	runing,     2000 points left at Tue Jan 23 15:24:11 2018
  thread-2	runing,     1000 points left at Tue Jan 23 15:24:37 2018
  thread-3	runing,        0 points left at Tue Jan 23 15:25:03 2018
1 stop2
 stop
5 stop6
 stop
0 stop
4 stop
3 stop
All points done. Use 0.209607 hours
3551 sample recorded in ./output/record_180123_152503


In [20]:
MTN.__dict__.keys()

dict_keys(['output_dir', 'input_mold', 'threads', 'work_space', 'package_mold', 'NT', 'sample_list', 'accepted_list', 'excluded_list', 'record_dir'])

In [21]:
MTN.record_dir

'./output/record_180121_143249'

In [22]:
len(MTN.accepted_list)

3670

In [23]:
acc=I2P(MTN.accepted_list,title='accepted')

In [26]:
acc[('flags','binary','1/0_is/not','calculable')]=1
acc.to_csv(os.path.join(MTN.output_dir,'accepted.csv'))

In [27]:
exc=I2P(MTN.excluded_list,title='excluded')
exc[('flags','binary','1/0_is/not','calculable')]=0
exc.to_csv(os.path.join(MTN.output_dir,'excluded.csv'))

In [28]:
from command.data_transformer.SpectrumToPandas import SpectrumToPandas as S2P
spc=S2P(MTN.sample_list)

width= {178}


In [31]:
mass=spc[('spectrum','MASS')][['25','35','36']]
mass.to_csv(os.path.join(MTN.output_dir,'mass.csv'))

In [32]:
MTN.sample_list[0].__dict__.keys()

dict_keys(['DECAY', 'SPINFO', 'MODSEL', 'SMINPUTS', 'MINPAR', 'EXTPAR', 'MASS', 'LOWEN', 'HMIX', 'NMHMIX', 'NMAMIX', 'STOPMIX', 'SBOTMIX', 'STAUMIX', 'NMNMIX', 'UMIX', 'VMIX', 'REDCOUP', 'HIGGSBOUNDSINPUTHIGGSCOUPLINGSBOSONS', 'HIGGSBOUNDSINPUTHIGGSCOUPLINGSFERMIONS', 'GAUGE', 'YU', 'YD', 'YE', 'AU', 'TU', 'AD', 'TD', 'AE', 'TE', 'MSOFT', 'NMSSMRUN', 'MSQ2', 'MSU2', 'MSD2', 'MSL2', 'MSE2', 'USQMIX', 'DSQMIX', 'SELMIX', 'GUTGAUGE', 'GUTYU', 'GUTYD', 'GUTYE', 'GUTAU', 'GUTAD', 'GUTAE', 'GUTMSOFT', 'GUTNMSSMRUN', 'FINETUNING', 'LHCCROSSSECTIONS', 'LHCFIT', 'EFFECTIVE_COUPLINGS', 'DCINFO', 'ERROR', 'constraints', 'documents'])

In [33]:
MTN.sample_list[0].constraints

['# Excluded by ee -> hZ, h -> bb',
 '# Excluded by ee -> hZ, h -> tautau',
 '# Excluded by ee -> hZ, h -> 2jets',
 '# Delta M_s more than 2 sigma away',
 '# Muon magn. mom. more than 2 sigma away',
 '# No Higgs in the 122.1-128.1 GeV mass range',
 '# Excluded by ggF->H/A->gamgam (ATLAS)',
 '# b -> c tau nu more than 2 sigma away (as SM)']